Stacking Classifier<br>
Author: Casey Kellett Whitwham


---

Imports

In [14]:
import importlib
import dataprep as dp
importlib.reload(dp)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

---

Load Processed Data

In [15]:
Xtrain,ytrain,Xtest,ytest,Xdev,ydev = dp.loadprocesseddata()

(24384, 12)
Train X,y shape:  (14609, 84) (14609,)
Test X,y shape:  (4876, 84) (4876,)
Dev X,y shape:  (4878, 84) (4878,)


---

Use top params from project 1's Decision Tree

In [16]:
top_dt ={'criterion': 'entropy','max_depth': 10,'max_features': None, 'min_samples_leaf': 4,'min_samples_split': 10, 'class_weight': None}
dt_fin =  DecisionTreeClassifier(**top_dt)

---

Ensuring Y sets are being passed as 1d arrays - Ran into setting invalid as nan warnings.

In [17]:
ytrain = np.ravel(ytrain)
ydev = np.ravel(ydev)

---

Define models for stacking (top 3 ensemble methods from project 2)

In [18]:
rf_fin = RandomForestClassifier(**top_dt,n_estimators=200)

top_dt_upd = top_dt.copy() 
top_dt_upd.pop('class_weight', None)
top_dt_upd.pop('criterion', None)
grad_fin = GradientBoostingClassifier(**top_dt_upd,learning_rate= 0.5,loss ='log_loss',n_estimators= 150)

hist_fin = HistGradientBoostingClassifier(l2_regularization = 0.0, learning_rate=  0.1, max_iter= 300)

bag_fin = BaggingClassifier(estimator=dt_fin,max_samples=1.0, n_estimators=50)

---
Create model

In [22]:
estimators = [('rf', rf_fin),('grad', grad_fin),('hist', hist_fin)]
stack_fin = StackingClassifier(estimators=estimators,final_estimator=LogisticRegression(),n_jobs=-1)

---
Evaluate Model

In [23]:
stack_fin.fit(Xtrain,ytrain)
y_pred = stack_fin.predict(Xtest)
accuracy = accuracy_score(ytest, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9315012305168171


c:\Users\casey\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
